<a href="https://colab.research.google.com/github/evacragnolino/Data-Science-Foundations/blob/main/EvaC_Unit5_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

How Well Does Cross Country Perfomance Predict a Track 5000m?

The BU Sharon Colyear- Danville Season Opener is a track meet held 2 weeks after the cross country national championships. A lot of coaches enter their athletes to recycle their cross country fitness to try to get qualifying time for the 5000m at the indoor track national championships. This works since the 6k cross country event and track 5k are pretty similar. Since the indoor season is very short and the national championships are vere selective, only 16 athletes per event qualify, it is hard to get enough training to compete well at nationals while also getting a qualifying time and competing at conference championships. Because of this many coaches opt to extend the cross country season two weeks into track so the athletes can get their qualyfying time out of the way and then take a break so they can focus on conference and nationals in the new year. Many athletes also use this as a chance to improve their times in longer events. This works since you are coming off cross country where you have done longer workouts and races.

In this project I used placing from cross country nationals to predict 5k times. There wasnt enough data from this year so I had to use data from 2024 as well which was not ideal.

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import arviz as az
import xarray as xr
import pymc as pm
import graphviz as gv

from scipy.interpolate import PchipInterpolator

In [2]:
url_xc = 'https://raw.githubusercontent.com/evacragnolino/Data-Science-Foundations/main/unit%205%20data%20set%20-%20Sheet1%20(2).csv'
xc = pd.read_csv(url_xc)

In [3]:
xc

,name,NCAA xc place,BU result
0,Jane Hedengren,2,14:44.79
1,Hilda Olemoimoi,3,15:08.61
2,Riley Chamberlain,4,14:58.97
3,Hannah Gapes,5,15:05.41
4,Mary Bonner Dalton,10,15:11.31
...,...,...,...
92,Jenna Zydanowicz,102,16:08.59
93,India Weir,113,15:57.36
94,Tilly Simpson,118,16:19.07
95,Julia Flynn,119,16:21.67


In [ ]:
import matplotlib.ticker as mticker

# Define a function to convert 'MM:SS.ms' string to total minutes (float)
def time_to_minutes(time_str):
    try:
        minutes, seconds_ms = time_str.split(':')
        total_minutes = float(minutes) + float(seconds_ms) / 60
        return total_minutes
    except ValueError:
        return float('nan') # Return NaN for unconvertible values

# Define a function to convert total minutes (float) back to 'MM:SS' string for axis labels
def minutes_to_mmss(x, pos):
    if pd.isna(x):
        return ""
    total_seconds = x * 60
    minutes = int(total_seconds // 60)
    seconds = int(total_seconds % 60)
    return f"{minutes:02d}:{seconds:02d}"

# Create a copy of the DataFrame to perform operations without modifying the original 'xc'
xc_plot_data = xc.copy()

# Convert the 'BU result ' column to a numerical representation in minutes
xc_plot_data['BU result (min)'] = xc_plot_data['BU result '].apply(time_to_minutes)

# Define y-axis limits based on user request (14:30 to 17:30)
y_min_str = "14:30"
y_max_str = "17:30"

y_min_minutes = time_to_minutes(y_min_str)
y_max_minutes = time_to_minutes(y_max_str)

# Create the scatter plot using the numerical y-axis data
ax = xc_plot_data.plot(x="NCAA xc place", y="BU result (min)", figsize=(12, 3), kind="scatter")

# Set the y-axis limits
ax.set_ylim(y_min_minutes, y_max_minutes)

# Apply a custom formatter to display y-axis ticks in 'MM:SS' format
formatter = mticker.FuncFormatter(minutes_to_mmss)
ax.yaxis.set_major_formatter(formatter)

# Optional: Add a descriptive label for the y-axis
ax.set_ylabel("BU Result (MM:SS)")

plt.tight_layout() # Adjust plot to ensure everything fits


Here I used Gemini to spread out the y axis labels. I could not figure out how to do it and my original code had all the the times on the y axis so it was unreadable.

In [ ]:
dag_xc = gv.Digraph(comment='xc_dag')

dag_xc.node('X', 'NCAA xc result')
dag_xc.node('B','BU result ')

dag_xc.edges(['XB', ])

dag_xc

In [ ]:
with pm.Model() as model_lxc:

    #priors for the linear part of our model
    α = pm.Normal("α", mu=0, sigma=100)
    β = pm.Normal("β", mu=0, sigma=10)

    #this is the linear part of our model
    μ = pm.Deterministic("μ", α + β * xc['NCAA xc place'])

    #prior for the standard deviation of our likelihood
    #Cauchy is a T dist with nu = 1
    σ = pm.HalfCauchy("σ", 10)

    #likelihood
    # Use the numerically converted 'BU result (min)' column from xc_plot_data
    y_pred = pm.Normal("y_pred", mu=μ, sigma=σ, observed=xc_plot_data['BU result (min)'])

    #inference data object
    idata_lxc = pm.sample()

In [ ]:
az.plot_posterior(idata_lxc, var_names=["~μ"], figsize=(12, 3))

In [ ]:
posterior = az.extract(idata_lxc, num_samples=100)

# grabbing x values for graphing.
x_plot = xr.DataArray(
    np.linspace(xc['NCAA xc place'].min(), xc['NCAA xc place'].max(), 50),
    dims="plot_id"
    )

# this creates the expected line, the path we predict temperature and
#rented bike count deviates from
mean_line = posterior["α"].mean() + posterior["β"].mean() * x_plot

#These are a 50 lines that our model came up with
lines = posterior["α"] + posterior["β"] * x_plot

#set up plot
_, ax = plt.subplots()

#plots 50 other lines our model came up with.
ax.plot(x_plot, lines.T, c="C1", alpha=0.2, label="lines")

#plots the mean line
ax.plot(x_plot, mean_line, c="C0", label="mean line")

#plot the raw data
ax.plot(xc['NCAA xc place'], xc_plot_data['BU result (min)'], "C2.", zorder=-3)

#label axes and create legend
ax.set_xlabel("NCAA xc place")
ax.set_ylabel("BU result (min)")

In [ ]:
pm.sample_posterior_predictive(idata_lxc, model=model_lxc,  extend_inferencedata=True)

In [ ]:
mean_line = idata_lxc.posterior["μ"].mean(("chain", "draw"))

#creates some x values to run through our line formula
# xc_place = np.random.normal(xc['NCAA xc place'].values, 0.01) # This variable was defined but not used for plotting the lines.
idx = np.argsort(xc['NCAA xc place'].values) # Sort indices based on 'NCAA xc place' for smooth plotting

# Create x values for interpolation (to get smoother curves for the HDI)
x = np.linspace(xc['NCAA xc place'].min(), xc['NCAA xc place'].max(), 50)

#grabs the 94% HDI and 50% HDI
y_pred_q = idata_lxc.posterior_predictive["y_pred"].quantile(
    [0.03, 0.97, 0.25, 0.75], dim=["chain", "draw"]
)

y_hat_bounds_list = []
# Iterate through each quantile (i represents the quantile index 0, 1, 2, 3)
for i in range(4):
    # Create a temporary DataFrame to pair 'NCAA xc place' with the current quantile's y-values
    temp_data = pd.DataFrame({
        'ncaa_xc_place': xc['NCAA xc place'].values,
        'y_quantile_values': y_pred_q[i].values
    })

    # Group by 'ncaa_xc_place' and calculate the mean of 'y_quantile_values'
    # This handles duplicate 'ncaa_xc_place' values by averaging their corresponding y-values,
    # ensuring unique x-values for the interpolator.
    grouped_data = temp_data.groupby('ncaa_xc_place')['y_quantile_values'].mean().reset_index()

    # Sort the grouped data by 'ncaa_xc_place' to ensure the x-values are strictly increasing,
    # which is a requirement for PchipInterpolator.
    grouped_data_sorted = grouped_data.sort_values(by='ncaa_xc_place')

    # Extract the unique, strictly increasing x-values and their corresponding mean y-values
    interp_x_unique = grouped_data_sorted['ncaa_xc_place'].values
    interp_y_for_quantile = grouped_data_sorted['y_quantile_values'].values

    # Create the PchipInterpolator using these unique and sorted x-values
    interpolator = PchipInterpolator(interp_x_unique, interp_y_for_quantile)

    # Evaluate the interpolator at the smooth `x` (linspace) for plotting
    y_hat_bounds_list.append(interpolator(x))

y_hat_bounds = iter(y_hat_bounds_list) # Make it an iterator as expected by the loop

#plots raw data and our line of best fit
_, ax = plt.subplots()
ax.plot(xc['NCAA xc place'], xc_plot_data['BU result (min)'], "C2.", zorder=-3) # Use the correct numerical 'BU result (min)'
ax.plot(xc['NCAA xc place'].values[idx], mean_line.values[idx], c="C0") # Plot sorted mean line


#graphs the 94% and 50% HDIs
for lb, ub in zip(y_hat_bounds, y_hat_bounds):
    ax.fill_between(x, lb, ub, color="C1", alpha=0.5)

#labels
ax.set_xlabel("NCAA xc place") # Corrected label
ax.set_ylabel("BU result (min)") # Corrected label

In [ ]:
az.plot_ppc(idata_lxc, num_pp_samples=200, colors=["C1", "C0", "C1"])

In [ ]:
import matplotlib.ticker as mticker

# Define a function to convert 'MM:SS.ms' string to total minutes (float)
def time_to_minutes(time_str):
    try:
        minutes, seconds_ms = time_str.split(':')
        total_minutes = float(minutes) + float(seconds_ms) / 60
        return total_minutes
    except ValueError:
        return float('nan') # Return NaN for unconvertible values

# Define a function to convert total minutes (float) back to 'MM:SS' string for axis labels
def minutes_to_mmss(x, pos):
    if pd.isna(x):
        return ""
    total_seconds = x * 60
    minutes = int(total_seconds // 60)
    seconds = int(total_seconds % 60)
    return f"{minutes:02d}:{seconds:02d}"

# Create a copy of the DataFrame to perform operations without modifying the original 'xc'
xc_plot_data = xc.copy()

# Convert the 'BU result ' column to a numerical representation in minutes
xc_plot_data['BU result (min)'] = xc_plot_data['BU result '].apply(time_to_minutes)

# Define y-axis limits based on user request (14:30 to 17:30)
y_min_str = "14:30"
y_max_str = "17:30"

y_min_minutes = time_to_minutes(y_min_str)
y_max_minutes = time_to_minutes(y_max_str)

# Create the violin plot using the numerical y-axis data
fig, ax = plt.subplots(figsize=(10, 5))
sns.violinplot(y=xc_plot_data['BU result (min)'], ax=ax, orient='v')

# Set the y-axis limits
ax.set_ylim(y_min_minutes, y_max_minutes)

# Apply a custom formatter to display y-axis ticks in 'MM:SS' format
formatter = mticker.FuncFormatter(minutes_to_mmss)
ax.yaxis.set_major_formatter(formatter)

# Optional: Add a descriptive label for the y-axis
ax.set_ylabel("BU Result (MM:SS)")

plt.title("Distribution of BU Results")
plt.tight_layout()
plt.show()

In [ ]:
with pm.Model() as model_sqrt_xc:

    #priors for the linear part of the model
    α = pm.Normal("α", mu=0, sigma=100)
    β = pm.Normal("β", mu=0, sigma=10)

    #the linear part of our model,
    #but with a twist:
    #our line is exponentiated, in order to make our all our values positive
    μ = pm.Deterministic("μ", pm.math.sqrt(α + β * xc['NCAA xc place']))

    #prior for the likelihood's standard deviation
    σ = pm.HalfNormal("σ", 10)

    #likelihood
    y_pred = pm.Normal("y_pred", mu=μ, sigma=σ, observed=xc_plot_data['BU result (min)'])

    idata_sqrt_xc = pm.sample()

In [26]:
pm.sample_posterior_predictive(idata_sqrt_xc, model = model_sqrt_xc, extend_inferencedata=True)

Output()

Inference data with groups:
	> posterior
	> posterior_predictive
	> sample_stats
	> observed_data

In [ ]:
az.plot_ppc(idata_sqrt_xc, num_pp_samples=200, colors=["C1", "C0", "C1"])

In [ ]:
# Use the same x_plot as in sDXFS2RUVAPY for consistency for the smooth curves
x_plot = np.linspace(xc['NCAA xc place'].min(), xc['NCAA xc place'].max(), 50)

# Calculate the mean of μ across chains and draws for each observed NCAA xc place
mean_mu_at_observed_x = idata_sqrt_xc.posterior["μ"].mean(("chain", "draw"))

# Interpolate the mean_mu_at_observed_x to get a smooth mean line over x_plot
mean_mu_interp_data = pd.DataFrame({
    'ncaa_xc_place': xc['NCAA xc place'].values,
    'mean_mu_values': mean_mu_at_observed_x.values
})
mean_mu_grouped_data = mean_mu_interp_data.groupby('ncaa_xc_place')['mean_mu_values'].mean().reset_index()
mean_mu_grouped_data_sorted = mean_mu_grouped_data.sort_values(by='ncaa_xc_place')
mean_mu_interp_x_unique = mean_mu_grouped_data_sorted['ncaa_xc_place'].values
mean_mu_interp_y_values = mean_mu_grouped_data_sorted['mean_mu_values'].values
mean_mu_interpolator = PchipInterpolator(mean_mu_interp_x_unique, mean_mu_interp_y_values)
smooth_mean_line_for_plot = mean_mu_interpolator(x_plot)


# Calculate quantiles of posterior predictive
y_pred_q = idata_sqrt_xc.posterior_predictive["y_pred"].quantile(
    [0.03, 0.97, 0.25, 0.75], dim=["chain", "draw"]
)

y_hat_bounds_list = []
for i in range(4):
    temp_data = pd.DataFrame({
        'ncaa_xc_place': xc['NCAA xc place'].values,
        'y_quantile_values': y_pred_q[i].values
    })
    grouped_data = temp_data.groupby('ncaa_xc_place')['y_quantile_values'].mean().reset_index()
    grouped_data_sorted = grouped_data.sort_values(by='ncaa_xc_place')
    interp_x_unique = grouped_data_sorted['ncaa_xc_place'].values
    interp_y_for_quantile = grouped_data_sorted['y_quantile_values'].values
    interpolator = PchipInterpolator(interp_x_unique, interp_y_for_quantile)
    y_hat_bounds_list.append(interpolator(x_plot))

y_hat_bounds_iterator = iter(y_hat_bounds_list)


_, ax = plt.subplots()

# Plot raw data
ax.plot(xc['NCAA xc place'], xc_plot_data['BU result (min)'], "C2.", zorder=-3)

# Plot the smooth mean line
ax.plot(x_plot, smooth_mean_line_for_plot, c="C0", label="Mean prediction")

# Graphs the 94% and 50% HDIs
# y_hat_bounds_list contains [q0.03, q0.97, q0.25, q0.75] in this order
lower_94, upper_94 = next(y_hat_bounds_iterator), next(y_hat_bounds_iterator)
lower_50, upper_50 = next(y_hat_bounds_iterator), next(y_hat_bounds_iterator)

ax.fill_between(x_plot, lower_94, upper_94, color="C1", alpha=0.5, label="94% HDI")
ax.fill_between(x_plot, lower_50, upper_50, color="C1", alpha=0.3, label="50% HDI")


ax.set_xlabel("NCAA xc place")
ax.set_ylabel("BU result (min)")
plt.legend()
plt.tight_layout()
plt.show()

Both of the lines  look pretty similar and they show alot of correlation which makes sense.

Conclusion:

Cross country predicts 5k times pretyy well which makes sense. The outliers come from people who are much better at either track or cross country or people who had a particularly bad race at either nationals or the season opener.